In [ ]:
from newspaper import Article
from tqdm import tqdm
import pandas as pd
from soynlp.normalizer import *
import re

In [ ]:
def preprocess(text):
    # HTML 태그 제거
    text = re.sub(r"<[^>]+>\s+(?=<)|<[^>]+>", "", text).strip()

    # 이메일 제거
    text = re.sub(r"[a-zA-Z0-9+-_.]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+", "", text).strip()

    # 해쉬태그 제거
    text = re.sub(r"#\S+", "", text).strip()

    # 멘션 제거
    text = re.sub(r"@\w+", "", text).strip()

    # URL 제거
    text = re.sub(r"(http|https)?:\/\/\S+\b|www\.(\w+\.)+\S*", "", text).strip()
    text = re.sub(r"pic\.(\w+\.)+\S*", "", text).strip()

    # 특수 문자 제거
    text = re.sub(r"[^\w\s…]", "", text)

    # 문제를 일으킬 수 있는 문자 제거
    bad_chars = {"\u200b": "", "…": " ... ", "\ufeff": ""}
    for bad_char in bad_chars:
        text = text.replace(bad_char, bad_chars[bad_char])
    text = re.sub(r"[\+á?\xc3\xa1]", "", text)

    # 언론 정보 제거
    re_patterns = [
        r"\([^(]*?(뉴스|경제|일보|미디어|데일리|한겨례|타임즈|위키트리)\)",
        r"[가-힣]{0,4} (기자|선임기자|수습기자|특파원|객원기자|논설고문|통신원|연구소장) ",  # 이름 + 기자
        r"[가-힣]{1,}(뉴스|경제|일보|미디어|데일리|한겨례|타임|위키트리)",  # (... 연합뉴스) ..
        r"\(\s+\)",  # (  )
        r"\(=\s+\)",  # (=  )
        r"\(\s+=\)",  # (  =)
    ]
    for re_pattern in re_patterns:
        text = re.sub(re_pattern, "", text).strip()

    # 저작권 관련 텍스트 제거
    re_patterns = [
        r"\<저작권자(\(c\)|ⓒ|©|\(Copyright\)|(\(c\))|(\(C\))).+?\>",
        r"저작권자\(c\)|ⓒ|©|(Copyright)|(\(c\))|(\(C\))"
    ]

    for re_pattern in re_patterns:
        text = re.sub(re_pattern, "", text).strip()

    # 사진 라벨 제거
    text = re.sub(r"\(출처 ?= ?.+\) |\(사진 ?= ?.+\) |\(자료 ?= ?.+\)| \(자료사진\) |사진=.+기자 ", "", text).strip()

    # 쓸모없는 괄호 제거
    bracket_pattern = re.compile(r"\((.*?)\)")
    modi_text = ""
    text = text.replace("()", "")  # 수학() -> 수학
    brackets = bracket_pattern.search(text)
    if brackets:
        replace_brackets = {}
        # key: 원본 문장에서 고쳐야하는 index, value: 고쳐져야 하는 값
        # e.g. {'2,8': '(數學)','34,37': ''}
        while brackets:
            index_key = str(brackets.start()) + "," + str(brackets.end())
            bracket = text[brackets.start() + 1 : brackets.end() - 1]
            infos = bracket.split(",")
            modi_infos = []
            for info in infos:
                info = info.strip()
                if len(info) > 0:
                    modi_infos.append(info)
            if len(modi_infos) > 0:
                replace_brackets[index_key] = "(" + ", ".join(modi_infos) + ")"
            else:
                replace_brackets[index_key] = ""
            brackets = bracket_pattern.search(text, brackets.start() + 1)
        end_index = 0
        for index_key in replace_brackets.keys():
            start_index = int(index_key.split(",")[0])
            modi_text += text[end_index:start_index]
            modi_text += replace_brackets[index_key]
            end_index = int(index_key.split(",")[1])
        modi_text += text[end_index:]
        modi_text = modi_text.strip()
        text = modi_text
    

    # 반복 문자 개수 고정
    text = repeat_normalize(text, num_repeats=2).strip()

    # 기호 치환
    punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-", "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', }
    for p in punct_mapping:
        text = text.replace(p, punct_mapping[p])
    text = text.strip()

    # 연속 공백 제거
    text = re.sub(r"\s+", " ", text).strip()

    return text

In [ ]:
train = pd.read_csv('./train.csv')

In [ ]:
texts = []
for news_url in tqdm(train['url']):
    try:
        article = Article(news_url, language='ko')
        article.download()
        article.parse()
    except:
        texts.append("except")
    texts.append(article.title)

In [ ]:
except_idx = [idx for idx, val in enumerate(texts) if val=="except"]

In [ ]:
train.iloc[except_idx]['text']

In [ ]:
texts[14065] = "한진해운 글로벌 해운동맹 합류…현대상선 일단 제외 종합2보"
texts[21419] = "81세 부테플리카 알제리 대통령 대선 출마 선언"
texts[24213] = "안테토쿤보 30점 ... 밀워키 보스턴 3연패 밀어 넣고 4연승"
texts[25874] = "朴대통령 내주초 추가 메시지 내놓을듯 ... 국민단합 호소종합"
texts[30201] = "삼성카드 핸드메이드 유통 플랫폼 백패커와 업무제휴"
texts[30553] = "수소위원회 만찬 후 기념촬영하는 3사 CEO"

In [ ]:
clean_texts = [preprocess(text) for text in texts]

In [ ]:
clean_data=pd.DataFrame({'texts':clean_texts})
clean_data.to_csv('./clean_data.csv', index=False) 